# PyTorch Deep Dive: Introduction and Tensors

Welcome to the absolute foundation of Deep Learning. 

Before we write code, we need to define the **Objects** we are working with.

## Learning Objectives
- **The Vocabulary**: What is a "Tensor", "Scalar", "Vector", and "Matrix"?
- **The Intuition**: Tensors as "Excel Sheets on Steroids".
- **The Hardware**: CPU vs GPU (Minivan vs Sports Car).
- **The Mechanics**: Shapes, Dtypes, and Broadcasting.
- **The Deep Dive**: Strides and Memory Layout (What actually happens in RAM).


In [6]:
import torch
import numpy as np
import matplotlib.pyplot as plt

print("PyTorch version:", torch.__version__)

PyTorch version: 2.3.0


## Part 1: The Vocabulary (Definitions First)

In Data Science, we have specific names for containers of numbers based on their dimensions.

### 1. Scalar (0D)
- A single number.
- Example: `7`, `3.14`.
- Use case: A loss value, a learning rate.

### 2. Vector (1D)
- A list of numbers.
- Example: `[1, 2, 3]`.
- Use case: A row of data, a bias term.

### 3. Matrix (2D)
- A grid of numbers (Rows and Columns).
- Example: An Excel sheet, a black-and-white image.
- Use case: A dataset, weights of a linear layer.

### 4. Tensor (ND)
- A generic term for N-dimensional arrays (3D, 4D, etc.).
- Example: A color image (Height x Width x 3 Channels), a video (Time x H x W x C).
- Use case: Complex data structures.

## Part 2: The Intuition (Excel on Steroids)

Now that we know the terms, let's visualize them.

- **2D Tensor**: Imagine a single Excel sheet.
- **3D Tensor**: Imagine a **Workbook** containing multiple Excel sheets.
- **4D Tensor**: Imagine a **Filing Cabinet** containing multiple Workbooks.
- **5D Tensor**: Imagine a **Library** containing multiple Filing Cabinets.

PyTorch Tensors are just these containers, but optimized for math.

In [7]:
# 0D Tensor (Scalar)
scalar = torch.tensor(7)
print(f"Scalar: {scalar.item()} (Shape: {scalar.shape})")

# 1D Tensor (Vector)
vector = torch.tensor([7, 2, 5])
print(f"Vector: {vector} (Shape: {vector.shape})")

# 2D Tensor (Matrix)
matrix = torch.tensor([[1, 2], [3, 4]])
print(f"Matrix:\n{matrix} (Shape: {matrix.shape})")

# 3D Tensor (Image-like)
tensor3d = torch.rand(3, 4, 4) # 3 Channels, 4x4 pixels
print(f"3D Tensor Shape: {tensor3d.shape}")

Scalar: 7 (Shape: torch.Size([]))
Vector: tensor([7, 2, 5]) (Shape: torch.Size([3]))
Matrix:
tensor([[1, 2],
        [3, 4]]) (Shape: torch.Size([2, 2]))
3D Tensor Shape: torch.Size([3, 4, 4])


## Part 3: The Hardware (Minivan vs Sports Car)

Why do we use PyTorch Tensors instead of Python lists or NumPy arrays?

**The GPU.**

- **CPU (Central Processing Unit)**: Like a **Minivan**. It can carry a few things (instructions) very quickly and flexibly. Good for sequential logic.
- **GPU (Graphics Processing Unit)**: Like a **Train** or **Fleet of Sports Cars**. It can carry MASSIVE amounts of data at once, but it's hard to turn. Good for parallel math (matrix multiplication).

PyTorch Tensors can live on the GPU. NumPy arrays cannot.

In [8]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if device.type == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Move tensor to GPU
x = torch.tensor([1, 2, 3])
x_gpu = x.to(device)

print(f"x is on: {x.device}")
print(f"x_gpu is on: {x_gpu.device}")

# Note: You cannot add a CPU tensor to a GPU tensor. They live in different worlds!
# x + x_gpu # This would error

Using device: cpu
x is on: cpu
x_gpu is on: cpu


## Part 4: The Mechanics (Broadcasting)

Broadcasting is magic. It allows you to do math on tensors of different shapes.

Imagine you have a matrix of students' scores (Rows=Students, Cols=Subjects).
You want to add 5 bonus points to **every** score.

You don't need to create a matrix of 5s. You just add `5`.
PyTorch "broadcasts" (stretches) the `5` to match the matrix shape.

In [9]:
matrix = torch.tensor([[10, 20, 30], 
                       [40, 50, 60]])

# Add scalar (Broadcasting 5 to match 2x3)
print("Matrix + 5:\n", matrix + 5)

# Add vector (Broadcasting [1, 2, 3] to every row)
vector = torch.tensor([1, 2, 3])
print("Matrix + Vector:\n", matrix + vector)

# Visualizing what happened:
# [10, 20, 30] + [1, 2, 3] = [11, 22, 33]
# [40, 50, 60] + [1, 2, 3] = [41, 52, 63]

Matrix + 5:
 tensor([[15, 25, 35],
        [45, 55, 65]])
Matrix + Vector:
 tensor([[11, 22, 33],
        [41, 52, 63]])


## Part 5: The Deep Dive (Strides & Memory)

Here is the secret: **A Tensor is just a 1D array in memory.**

The "shape" (rows, columns) is an illusion created by **Strides**.

- **Stride**: How many steps in memory do I need to jump to get to the next element in this dimension?

Example: Matrix 2x3
`[[1, 2, 3], [4, 5, 6]]`

In memory (RAM): `[1, 2, 3, 4, 5, 6]`

To go from `1` to `2` (next column), jump 1 step.
To go from `1` to `4` (next row), jump 3 steps.

Stride = `(3, 1)`.

In [10]:
t = torch.tensor([[1, 2, 3], 
                  [4, 5, 6]])

print(f"Shape: {t.shape}")
print(f"Stride: {t.stride()}")

# Transpose it (swap rows/cols)
t_T = t.t()
print(f"\nTransposed Shape: {t_T.shape}")
print(f"Transposed Stride: {t_T.stride()}")

# Notice: The DATA in memory didn't move! PyTorch just swapped the strides.
# This makes transposing instant, even for huge matrices.

Shape: torch.Size([2, 3])
Stride: (3, 1)

Transposed Shape: torch.Size([3, 2])
Transposed Stride: (1, 3)


## Part 6: Advanced Tensor Operations (FAANG Interview Essentials)

### Views vs Copies - Critical Interview Topic

**View**: A new tensor object that shares the same underlying data.
**Copy**: A completely independent tensor with its own memory.

This distinction is CRITICAL for:
1. Memory efficiency (views are nearly free)
2. Bug prevention (modifying views affects originals)
3. Understanding when `contiguous()` is needed

In [ ]:
# Views vs Copies - FAANG Interview Classic

original = torch.tensor([1, 2, 3, 4, 5, 6])

# View: shares memory with original
view = original.view(2, 3)
print(f"Original: {original}")
print(f"View:\n{view}")
print(f"Same memory? {original.data_ptr() == view.data_ptr()}")  # True!

# Modify the view - affects original!
view[0, 0] = 999
print(f"\nAfter modifying view[0,0] = 999:")
print(f"Original: {original}")  # Also changed!
print(f"View:\n{view}")

# Clone: creates independent copy
original = torch.tensor([1, 2, 3, 4, 5, 6])
copy = original.clone()
copy[0] = 888
print(f"\nAfter cloning and modifying copy[0] = 888:")
print(f"Original: {original}")  # Unchanged!
print(f"Copy: {copy}")

print("\n🔑 FAANG Interview Tip: Always use .clone() when you need to modify a tensor independently!")

### Contiguous Memory - Why `.contiguous()` Matters

After certain operations (like transpose), the memory layout becomes non-contiguous.
Some operations (like `.view()`) require contiguous memory.

**Interview Question**: "When would you need to call `.contiguous()`?"

In [ ]:
# Contiguous Memory Demo

t = torch.tensor([[1, 2, 3],
                  [4, 5, 6]])

print(f"Original is contiguous: {t.is_contiguous()}")
print(f"Original stride: {t.stride()}")

# Transpose creates a non-contiguous view
t_transposed = t.t()
print(f"\nTransposed is contiguous: {t_transposed.is_contiguous()}")
print(f"Transposed stride: {t_transposed.stride()}")

# This will fail:
try:
    t_transposed.view(6)  # Can't view non-contiguous tensor!
except RuntimeError as e:
    print(f"\n❌ Error: {e}")

# Solution: Make it contiguous first
t_contiguous = t_transposed.contiguous()
print(f"\nAfter .contiguous(): {t_contiguous.is_contiguous()}")
print(f"Now we can view: {t_contiguous.view(6)}")

# BETTER: Use .reshape() instead - it handles this automatically!
print(f"\n✅ reshape() handles it: {t_transposed.reshape(6)}")

print("\n🔑 FAANG Tip: Use .reshape() over .view() for safer code. View is faster but reshape handles non-contiguous.")

### In-Place Operations - Memory Efficiency at Scale

In-place operations modify tensors directly without creating copies.
Denoted by underscore suffix: `add_()`, `mul_()`, `zero_()`, etc.

**Trade-offs**:
- ✅ Saves memory (critical for large models)
- ❌ Can break autograd computation graphs
- ❌ Harder to debug

In [ ]:
# In-Place Operations Demo

x = torch.tensor([1.0, 2.0, 3.0])
print(f"Original x: {x}, id: {id(x)}")

# Out-of-place (creates new tensor)
y = x + 10
print(f"After x + 10: y = {y}, x = {x}")
print(f"Same object? {id(x) == id(y)}")  # False

# In-place (modifies x directly)
x.add_(10)
print(f"\nAfter x.add_(10): x = {x}")

# Common in-place operations
x = torch.tensor([1.0, 2.0, 3.0])
x.mul_(2)      # x = x * 2
print(f"After mul_(2): {x}")

x.zero_()      # x = 0
print(f"After zero_(): {x}")

x.fill_(42)    # x = 42
print(f"After fill_(42): {x}")

# DANGER: In-place on tensors with requires_grad can cause issues!
x = torch.tensor([1.0, 2.0], requires_grad=True)
y = x * 2
try:
    x.add_(1)  # This breaks the computation graph!
except RuntimeError as e:
    print(f"\n❌ In-place error with gradients: {str(e)[:80]}...")

print("\n🔑 FAANG Tip: Avoid in-place ops during forward pass. Use them for weight updates or inference only.")

## Part 7: Tensor Creation Patterns (Production Code)

Different creation methods for different scenarios. Knowing these shows production experience.

In [ ]:
# Production Tensor Creation Patterns

# 1. Zeros/Ones - For initializing buffers
zeros = torch.zeros(3, 4)                    # All zeros
ones = torch.ones(3, 4)                      # All ones
full = torch.full((3, 4), fill_value=3.14)   # All same value

# 2. Like methods - Match device/dtype of existing tensor (CRITICAL for multi-GPU)
x = torch.randn(2, 3, device='cpu', dtype=torch.float32)
zeros_like = torch.zeros_like(x)  # Same device, dtype, shape!
ones_like = torch.ones_like(x)

# 3. Random initialization
uniform = torch.rand(3, 4)          # Uniform [0, 1)
normal = torch.randn(3, 4)          # Normal (mean=0, std=1)
randint = torch.randint(0, 10, (3, 4))  # Random integers

# 4. Ranges
arange = torch.arange(0, 10, 2)     # [0, 2, 4, 6, 8]
linspace = torch.linspace(0, 1, 5)  # [0, 0.25, 0.5, 0.75, 1.0]

# 5. Identity matrix (for attention masks, etc.)
eye = torch.eye(4)

# 6. Diagonal matrix
diag = torch.diag(torch.tensor([1, 2, 3]))

# 7. Empty (FAST but DANGEROUS - contains garbage!)
empty = torch.empty(3, 4)  # Faster than zeros, but uninitialized

print("Creation patterns demo:")
print(f"zeros_like matches device: {zeros_like.device == x.device}")
print(f"linspace: {linspace}")
print(f"eye:\n{eye}")

print("\n🔑 FAANG Tip: Always use *_like() methods in multi-GPU code to ensure device consistency!")

## Part 8: Common Tensor Operations (Interview Favorites)

These operations appear in almost every ML codebase and interview.

In [ ]:
# Common Tensor Operations - Know These Cold!

x = torch.tensor([[1, 2, 3],
                  [4, 5, 6]], dtype=torch.float32)

# === Dimension Operations ===
print("=== Dimension Operations ===")
print(f"Sum all: {x.sum()}")
print(f"Sum along dim 0 (columns): {x.sum(dim=0)}")  # [5, 7, 9]
print(f"Sum along dim 1 (rows): {x.sum(dim=1)}")     # [6, 15]
print(f"Mean: {x.mean()}")
print(f"Max: {x.max()}, Argmax: {x.argmax()}")       # Value and index

# === Squeeze/Unsqueeze (add/remove dimensions) ===
print("\n=== Squeeze/Unsqueeze ===")
y = torch.tensor([1, 2, 3])
print(f"Original shape: {y.shape}")              # [3]
y_unsqueezed = y.unsqueeze(0)                    # Add dim at position 0
print(f"After unsqueeze(0): {y_unsqueezed.shape}")  # [1, 3]
y_unsqueezed2 = y.unsqueeze(1)                   # Add dim at position 1
print(f"After unsqueeze(1): {y_unsqueezed2.shape}") # [3, 1]

z = torch.zeros(1, 3, 1)
print(f"\nBefore squeeze: {z.shape}")            # [1, 3, 1]
print(f"After squeeze(): {z.squeeze().shape}")  # [3] - removes ALL size-1 dims
print(f"After squeeze(0): {z.squeeze(0).shape}") # [3, 1] - removes only dim 0

# === Concatenation vs Stacking ===
print("\n=== Cat vs Stack ===")
a = torch.tensor([[1, 2], [3, 4]])
b = torch.tensor([[5, 6], [7, 8]])

cat_result = torch.cat([a, b], dim=0)  # Concatenate along existing dim
print(f"cat dim=0 shape: {cat_result.shape}")  # [4, 2]

stack_result = torch.stack([a, b], dim=0)  # Create NEW dimension
print(f"stack dim=0 shape: {stack_result.shape}")  # [2, 2, 2]

# === Permute (reorder dimensions) ===
print("\n=== Permute ===")
img = torch.randn(3, 224, 224)  # CHW format (PyTorch)
img_hwc = img.permute(1, 2, 0)  # Convert to HWC (for matplotlib/PIL)
print(f"CHW: {img.shape} -> HWC: {img_hwc.shape}")

# === Flatten ===
print("\n=== Flatten ===")
x = torch.randn(2, 3, 4)
print(f"Before flatten: {x.shape}")
print(f"flatten(): {x.flatten().shape}")           # [24]
print(f"flatten(1): {x.flatten(1).shape}")         # [2, 12] - keep batch dim

print("\n🔑 FAANG Tip: Know dim=0 vs dim=1 - 90% of shape bugs are dimension mismatches!")

## Part 9: FAANG Interview Questions - Tensors & Memory

### Question 1: "Explain the difference between .view() and .reshape()"

**Answer**:
- `.view()` returns a view (shared memory) but REQUIRES contiguous memory
- `.reshape()` returns a view when possible, but copies if not contiguous
- `.view()` is slightly faster but less safe
- Use `.reshape()` in production for robustness

### Question 2: "What are strides in PyTorch? Why do they matter?"

**Answer**:
Strides define how many elements to skip in memory to move one step in each dimension.
- Enables zero-copy transpose, reshape, and slicing
- Non-contiguous tensors have different stride patterns
- Critical for understanding memory layout optimization

### Question 3: "How would you efficiently handle a batch of images on GPU?"

**Answer**:
```python
# 1. Create tensor directly on GPU
images = torch.randn(32, 3, 224, 224, device='cuda')

# 2. Or transfer with non-blocking (overlap CPU/GPU work)
images = images.to('cuda', non_blocking=True)

# 3. Use pinned memory for faster CPU->GPU transfer
images = torch.randn(32, 3, 224, 224).pin_memory()
images_gpu = images.to('cuda', non_blocking=True)
```

### Question 4: "What's the difference between torch.tensor() and torch.Tensor()?"

**Answer**:
- `torch.tensor()` - ALWAYS creates a new copy, infers dtype from data
- `torch.Tensor()` - May share memory, always creates float32
- Best practice: Always use `torch.tensor()` for explicit, predictable behavior

### Question 5: "How do you debug shape mismatches?"

**Answer**:
```python
# 1. Print shapes at each step
print(f"After conv: {x.shape}")

# 2. Use named tensors (PyTorch 1.4+)
x = x.refine_names('batch', 'channel', 'height', 'width')

# 3. Use einops for clarity
from einops import rearrange
x = rearrange(x, 'b c h w -> b (c h w)')
```

## Summary: Tensor Mastery Checklist

### Foundation (Know These Cold)
- [ ] Tensor = N-dimensional array (Scalar, Vector, Matrix, ND Tensor)
- [ ] GPU vs CPU: `.to(device)`, `.cuda()`, `.cpu()`
- [ ] Data types: `float32` (default), `float16` (mixed precision), `int64` (indices)
- [ ] Broadcasting: automatic shape expansion for element-wise ops

### Memory (Interview Favorites)
- [ ] Views vs Copies: `.view()` shares memory, `.clone()` copies
- [ ] Contiguous: `.is_contiguous()`, `.contiguous()`, stride patterns
- [ ] In-place: underscore suffix (`add_()`, `mul_()`) - use carefully with autograd

### Operations (Daily Usage)
- [ ] Reshape: `.view()`, `.reshape()`, `.flatten()`, `.squeeze()`, `.unsqueeze()`
- [ ] Combine: `torch.cat()` (along dim), `torch.stack()` (new dim)
- [ ] Reduce: `.sum()`, `.mean()`, `.max()` with `dim` argument
- [ ] Permute: `.permute()`, `.transpose()` for dimension reordering

### Production Patterns
- [ ] Use `*_like()` for device/dtype consistency
- [ ] Pin memory for fast CPU->GPU transfer
- [ ] Prefer `.reshape()` over `.view()` for safety
- [ ] Use `torch.no_grad()` for inference (we'll cover this next)

---
**Next**: Notebook 01 - Autograd and Gradients (How PyTorch computes derivatives)